Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic model

In [16]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  # 
  theta = tf.constant(0.001)
  l2_regular = tf.nn.l2_loss(weights)
  loss = loss + theta*l2_regular
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.447426
Minibatch accuracy: 14.8%
Validation accuracy: 15.5%
Minibatch loss at step 500: 2.735507
Minibatch accuracy: 74.2%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.450688
Minibatch accuracy: 79.7%
Validation accuracy: 78.1%
Minibatch loss at step 1500: 1.282835
Minibatch accuracy: 82.0%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.886729
Minibatch accuracy: 79.7%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 0.842283
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 0.686425
Minibatch accuracy: 83.6%
Validation accuracy: 81.6%
Test accuracy: 88.5%


### Neural Network Model

In [133]:
batch_size = 128
n_hiddens = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Hidden Layer: 1024 hidden units with relu activation.
  hidden_units = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hiddens]))
  hidden_biases = tf.Variable(tf.zeros([n_hiddens]))
  hidden_out = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_units) + hidden_biases)
  
  # Output Layer : .
  output_weights =  tf.Variable(
    tf.truncated_normal([n_hiddens, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_out, output_weights) + output_biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # 
  theta = tf.constant(0.001)
  l2_regular = tf.nn.l2_loss(hidden_units) + tf.nn.l2_loss(output_weights)
  loss = loss + theta*l2_regular
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_units) + hidden_biases), output_weights) + output_biases)
  test_prediction =  tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_units) + hidden_biases), output_weights) + output_biases)

In [134]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 675.065918
Minibatch accuracy: 7.8%
Validation accuracy: 31.6%
Minibatch loss at step 500: 196.863403
Minibatch accuracy: 80.5%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 114.612457
Minibatch accuracy: 82.8%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 69.610382
Minibatch accuracy: 83.6%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 41.485584
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.317734
Minibatch accuracy: 84.4%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 15.453297
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Test accuracy: 93.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [35]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step % 10 * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 635.909546
Minibatch accuracy: 7.0%
Validation accuracy: 30.3%
Minibatch loss at step 500: 190.515900
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 115.529076
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 70.060196
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 42.487873
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 25.768219
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 15.630158
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Test accuracy: 85.6%


```py
offset = (step % 20 * batch_size) % (train_labels.shape[0] - batch_size)
```
Restrict my training data to just the first **10** batches. From the training log, it can be seen that it got a 100% accuracy rate on minibatch fastly. And the validation and test accuracy decreased signigicantly.  
So, this model is ***overfitting***.


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### Neural network with Dropout

In [107]:
batch_size = 128
n_hiddens = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Hidden Layer: 1024 hidden units with relu activation.
  hidden_units = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hiddens]))
  hidden_biases = tf.Variable(tf.zeros([n_hiddens]))
  hidden_out = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_units) + hidden_biases)

  # Introduce Dropout on the hidden layer 
  hidden_dropout = tf.nn.dropout(hidden_out,keep_prob=0.5)
  
  # Output Layer : .
  output_weights =  tf.Variable(
    tf.truncated_normal([n_hiddens, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))
  
  # use hidden_dropout
  logits = tf.matmul(hidden_dropout, output_weights) + output_biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # 
  theta = tf.constant(0.001)
  l2_regular = tf.nn.l2_loss(hidden_units) + tf.nn.l2_loss(output_weights)
  loss = loss + theta*l2_regular
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # Dropout should only be introduced during training, so here just use hidden_out, omit the dropout operation
  train_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, hidden_units) + hidden_biases), output_weights) + output_biases)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_units) + hidden_biases), output_weights) + output_biases)
  test_prediction =  tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_units) + hidden_biases), output_weights) + output_biases)

In [108]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 503.207947
Minibatch accuracy: 7.8%
Validation accuracy: 33.5%


KeyboardInterrupt: 

### Extreme overfitting case

In [46]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step % 10 * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 810.270691
Minibatch accuracy: 7.8%
Validation accuracy: 34.2%
Minibatch loss at step 500: 192.786285
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 117.107826
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 71.351044
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 43.112835
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 26.170025
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 15.862151
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Test accuracy: 88.0%


With **Dropout** operation, the test accuracy decreased from ***92.8%*** to ***91.6%***.  
But in extreme overfitting case, the test accuracy increased from ***85.6%*** to ***88.0%***.  
**DropOut** is effective for **overfitting**

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [79]:
def cal_loss(logits, labls, theta = None, weights = None):
    """Calculate the loss from the logits and the labels
    
    Parameters:
        logits : networks's output with dropout
        labels : training datas' labels
        
    Returns:
        loss  : networks;s
        
    """
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labls))
    if theta:
        theta = tf.constant(theta)
        l2_regular = tf.nn.l2_loss(weights[0])
        for w in weights[1:]:
            l2_regular += tf.nn.l2_loss(w)
        loss = loss + theta*l2_regular
    return loss

In [176]:
def training(loss, learning_rate):
    """Setup the training Operation
    
    Parameters:
        loss : newworks' loss 
        learning_rate: learning rate to use for gradient descent.
        
    Returns:
        train_op : operation for training this network
    """
    # count the number of steps taken.
    global_step = tf.Variable(0)  
    #  learning rate decay
    learning_rate = tf.train.exponential_decay(learning_rate, global_step, 2000, 0.8)
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss)
    
    return train_op

The following code will produce a different result (1 hidden layer case).  
The difference is 
```py
    outputs = tf.nn.relu(tf.matmul(inputs, weights[0]) + biases[0])
    valid_prediction = tf.nn.relu(tf.matmul(valid, weights[0]) + biases[0])
    test_prediction = tf.nn.relu(tf.matmul(test, weights[0]) + biases[0])
```
Here, the code for computing outputs of train batch, valid data and test data are put together.  
But **why**?

```py
def build_model(inputs, n_inputs, n_layers, layers_units, n_outputs, valid, test,  keep_prob = 0.5):
    """Build a tensorflow network from parameters:
    
    Args:
        input:   networks's input
        layers_units : [[n_hidden_units, dropout(True/Fales)], [], [] ,,,]
        
    Returns:
        logits  : return networks's output(with dropout) for training
        outputs : return networks's output(without dropout) for evaluation
    """
    weights, biases = [], []
    
    # Hidden Layer: 1024 hidden units with relu activation.
    weights.append(tf.Variable(tf.truncated_normal([n_inputs, layers_units[0][0]])))
    biases.append(tf.Variable(tf.zeros([n_hiddens])))
    
    outputs = tf.nn.relu(tf.matmul(inputs, weights[0]) + biases[0])
    valid_prediction = tf.nn.relu(tf.matmul(valid, weights[0]) + biases[0])
    test_prediction = tf.nn.relu(tf.matmul(test, weights[0]) + biases[0])
    
    # Output Layer : .
    weights.append(tf.Variable(tf.truncated_normal([layers_units[-1][0], n_outputs])))
    biases.append(tf.Variable(tf.zeros([n_outputs])))    
    
    logits = tf.matmul(outputs, weights[-1]) + biases[-1]
    valid_prediction = tf.matmul(valid_prediction, weights[-1]) + biases[-1]
    test_prediction = tf.matmul(test_prediction, weights[-1]) + biases[-1]


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(valid_prediction)
    test_prediction =  tf.nn.softmax(test_prediction)

    return logits, train_prediction, valid_prediction, test_prediction
    
```

In [179]:
def build_model(inputs, n_inputs, n_layers, layers_units, n_outputs, valid, test,  keep_prob = 0.5):
    """Build a tensorflow network from parameters:
    
    Args:
        input:   networks's input
        layers_units : [[n_hidden_units, dropout(True/Fales)], [], [] ,,,]
        
    Returns:
        logits  : return networks's output(with dropout) for training
        outputs : return networks's output(without dropout) for evaluation
    """
    weights, biases = [], []
    
    # Hidden Layer:  input -- hidden
    weights.append(tf.Variable(tf.truncated_normal([n_inputs, layers_units[0][0]])))
    biases.append(tf.Variable(tf.zeros([layers_units[0][0]])))
    
    outputs = tf.nn.relu(tf.matmul(inputs, weights[0]) + biases[0])
    if layers_units[0][1]:
        outputs = tf.nn.dropout(outputs,keep_prob)
        
    # Hidden layers : hidden -- hidden
    for i in range(1,n_layers):
        # fectch parameters
        n_hidden_units_last, (n_hidden_units_current, isDropout) = layers_units[i-1][0], layers_units[i]

        # define variables        
        weights.append(tf.Variable(tf.truncated_normal([n_hidden_units_last, n_hidden_units_current])))
        biases.append(tf.Variable(tf.zeros([n_hidden_units_current])))

        # computatation
        outputs = tf.nn.relu(tf.matmul(outputs, weights[i]) + biases[i])

        # if this layer need dropout
        if isDropout:            
            outputs = tf.nn.dropout(outputs,keep_prob)
    
    # Output Layer : .
    weights.append(tf.Variable(tf.truncated_normal([layers_units[-1][0], n_outputs])))
    biases.append(tf.Variable(tf.zeros([n_outputs])))
    
    logits = tf.matmul(outputs, weights[-1]) + biases[-1]


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction, test_prediction = valid, test
    for w, b in zip(weights[:-1], biases[:-1]):
        valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, w) + b)
        test_prediction = tf.nn.relu(tf.matmul(test_prediction, w) + b)
    valid_prediction = tf.matmul(valid_prediction, weights[-1]) + biases[-1]
    test_prediction  = tf.matmul(test_prediction, weights[-1]) + biases[-1]
    valid_prediction = tf.nn.softmax(valid_prediction)
    test_prediction  =  tf.nn.softmax(test_prediction)

    return logits, train_prediction, valid_prediction, test_prediction, weights, biases
    

Notice : **A big learning rate will make NaN error**

In [180]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(123)
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # configutation 
    n_inputs = image_size * image_size
    
    n_layers = 2
    layer_units = [[512, False],[512, False]]
    
    n_outputs = num_labels
    learning_rate =  0.05
    
    # construct network
    logits, outputs, valid_outputs, test_outputs, weights, biases = build_model(tf_train_dataset, n_inputs, n_layers, layer_units, n_outputs, tf_valid_dataset, tf_test_dataset)
    loss =  cal_loss(logits, tf_train_labels, theta=0.001, weights=weights)
    optimizer = training(loss, learning_rate) 
                                    
    

In [178]:
num_steps = 50000
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, outputs], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_outputs.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_outputs.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3918.007812
Minibatch accuracy: 6.2%
Validation accuracy: 22.0%
Minibatch loss at step 500: 267.642517
Minibatch accuracy: 43.0%
Validation accuracy: 45.6%
Minibatch loss at step 1000: 251.596863
Minibatch accuracy: 56.2%
Validation accuracy: 54.0%
Minibatch loss at step 1500: 239.991989
Minibatch accuracy: 62.5%
Validation accuracy: 59.9%
Minibatch loss at step 2000: 227.303238
Minibatch accuracy: 64.8%
Validation accuracy: 61.3%
Minibatch loss at step 2500: 216.257507
Minibatch accuracy: 61.7%
Validation accuracy: 65.6%
Minibatch loss at step 3000: 205.643967
Minibatch accuracy: 63.3%
Validation accuracy: 67.0%
Minibatch loss at step 3500: 195.455917
Minibatch accuracy: 71.1%
Validation accuracy: 68.8%
Minibatch loss at step 4000: 186.030441
Minibatch accuracy: 66.4%
Validation accuracy: 69.3%
Minibatch loss at step 4500: 176.943741
Minibatch accuracy: 75.0%
Validation accuracy: 71.2%
Minibatch loss at step 5000: 168.330673
Minibatch accuracy: 76

Log:  

Fixed learning rate:
layer_units = [[1024, False]]  
Test accuracy: 92.9%

layer_units = [[512, False],[512, False]]  
learning_rate =  0.05  
loss =  cal_loss(logits, tf_train_labels, theta=0.001, weights=weights)  
Test accuracy: 94.1%